In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('HousingData.csv')

In [3]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2


In [4]:
df.shape

(506, 14)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     486 non-null    float64
 1   ZN       486 non-null    float64
 2   INDUS    486 non-null    float64
 3   CHAS     486 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      486 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    486 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [6]:
df.fillna(df.mean(),inplace=True)

In [7]:
X = df.iloc[:,0:13]
y = df.iloc[:,-1]

In [8]:
X.shape

(506, 13)

In [9]:
y.shape

(506,)

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=123)
print('Train/Test Sets Sizes : ',X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Train/Test Sets Sizes :  (404, 13) (102, 13) (404,) (102,)


In [12]:
lr = LinearRegression()
dt = DecisionTreeRegressor()
knn = KNeighborsRegressor()

In [13]:
lr.fit(X_train,y_train)
dt.fit(X_train,y_train)
knn.fit(X_train,y_train)

KNeighborsRegressor()

In [14]:
y_pred1 = lr.predict(X_test)
y_pred2 = dt.predict(X_test)
y_pred3 = knn.predict(X_test)

In [15]:
print('r2 Score of LR: ',r2_score(y_test,y_pred1))
print('r2 Score of DT: ',r2_score(y_test,y_pred2))
print('r2 Score of KNN: ',r2_score(y_test,y_pred3))

r2 Score of LR:  0.6418253578104465
r2 Score of DT:  0.20666635067522277
r2 Score of KNN:  0.48431793665022116


In [16]:
from sklearn.ensemble import BaggingRegressor

In [17]:
bag_reg = BaggingRegressor(random_state=1)
bag_reg.fit(X_train,y_train)

BaggingRegressor(random_state=1)

In [18]:
y_preds = bag_reg.predict(X_test)

In [19]:
print('Training Coefficient of R^2 : %.3f'%bag_reg.score(X_train, y_train))
print('Test Coefficient of R^2 : %.3f'%bag_reg.score(X_test, y_test))

Training Coefficient of R^2 : 0.978
Test Coefficient of R^2 : 0.690


In [21]:
# Define the BaggingRegressor with DecisionTreeRegressor as the base estimator
bagging = BaggingRegressor(
    estimator=DecisionTreeRegressor(),  # Corrected from base_estimator to estimator
    n_jobs=-1,
    random_state=1
)

# Define hyperparameter grid
param_grid = {
    'n_estimators': [10, 50, 100],  # Number of trees in the ensemble
    'max_samples': [0.5, 0.7, 1.0],  # Proportion of training samples
    'max_features': [0.5, 0.7, 1.0]  # Proportion of features
}

# Perform GridSearchCV
grid_search = GridSearchCV(bagging, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=5,
             estimator=BaggingRegressor(estimator=DecisionTreeRegressor(),
                                        n_jobs=-1, random_state=1),
             n_jobs=-1,
             param_grid={'max_features': [0.5, 0.7, 1.0],
                         'max_samples': [0.5, 0.7, 1.0],
                         'n_estimators': [10, 50, 100]},
             verbose=1)

In [24]:
from sklearn.metrics import mean_squared_error
best_model = grid_search.best_estimator_

# Evaluate on the test set
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)  # Now mean_squared_error is correctly imported

# Print results
print("Best Parameters:", grid_search.best_params_)
print("Test MSE:", mse)

Best Parameters: {'max_features': 0.7, 'max_samples': 1.0, 'n_estimators': 100}
Test MSE: 15.741157284313728
